In [1]:
import os
import numpy as np

from src.larp import LARP
from src.dow import DOW

from src.get_data import random_data
from src.dows_generator import dows_generator

In [2]:
k_vehicles = 2
Q_vehicle_capacity = 100
facility = 'F'

m_storages = 4
n_fields = 10

folder_path = os.getcwd() + '\\backup\\waterflow\\'

use = (n_fields, m_storages, folder_path)
fields, storages, households, f, q, fs_dist, cs_dist, d = random_data(*use)

Folder already exists: c:\Users\pierp\OneDrive\Desktop\Optimization Models\Python exercises\DSSC_MO\backup\waterflow\


In [3]:
larp_model = LARP(facility, 
                k_vehicles, 
                Q_vehicle_capacity, 
                fields, 
                storages, 
                households, 
                f, 
                q, 
                fs_dist, 
                cs_dist, 
                d)

# NOTE: Since it is difficult to find an optimal solution 
# trying with random X, Y and Z (decision variables)
# we fix a random X and set guroby to find and stop the very first feasible solution
larp_model.model.setParam('SolutionLimit', 1)
larp_model.build()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-12
LARP decision variables defined
LARP objective function defined
LARP constrains defined
-- LARP model build COMPLETED --


In [4]:
n_dows = 1
generator = dows_generator(larp_model, n_dows, m_storages, n_fields, k_vehicles)
rainfall = list()
for i, dow in enumerate(generator):
    print('i:', i, 'objvalue:', dow.obj_value, sep=' ')
    rainfall.append(dow)

generator loop: 0
dow created
LARP model optimized
LARP status : 10
iter: 0  --> DOW FEASIBLE
i: 0 objvalue: 143030676.0


In [5]:
dow = rainfall[0]

In [6]:
print(dow.X)

[1 0 1 1]


In [7]:
print(dow.Y)

[[-0.  0.  1. -0.]
 [-0.  0.  1. -0.]
 [-0.  0.  1. -0.]
 [ 1.  0. -0. -0.]
 [-0.  0. -0.  1.]
 [-0.  0.  1. -0.]
 [-0.  0. -0.  1.]
 [ 1.  0. -0. -0.]
 [ 1.  0. -0. -0.]
 [ 1.  0. -0. -0.]]


In [8]:
print(dow.Z)

[[ 0.  0. -0. -0.  1.]
 [ 0.  0.  0.  0. -0.]
 [-0.  0.  0.  1. -0.]
 [-0.  0. -0.  0.  1.]
 [ 1. -0.  1. -0.  0.]]


In [9]:
from src.opt_1_neighbourhood import opt_1

local_optimum = opt_1(larp_model, dow)
print(local_optimum)

idx: 0
idx: 1
objValue: 7.777820083333348e+18

X: [1 1 1 1]
Y: [4 4 4 4 2 3 2 4 2 2]
Z: [0 1 5 0 3 4 2 5]

objValue: 820083333341661.0

X: [1 1 1 1]
Y: [4 4 4 2 4 3 2 4 2 2]
Z: [0 1 5 0 3 4 2 5]

objValue: 7.778496744577789e+18

X: [1 1 1 1]
Y: [4 4 4 1 1 4 3 1 2 4]
Z: [0 1 5 0 3 4 2 5]

objValue: 7.778486744577778e+18

X: [1 1 1 1]
Y: [4 4 4 1 1 4 2 1 3 4]
Z: [0 1 5 0 3 4 2 5]

objValue: 7.778477861120009e+18

X: [1 1 1 1]
Y: [4 4 3 4 2 4 2 4 2 2]
Z: [0 1 5 0 3 4 2 5]

objValue: 1.3333745222987794e+18

X: [1 1 1 1]
Y: [4 4 3 3 3 1 4 2 2 3]
Z: [0 1 5 0 3 4 2 5]

objValue: 40083342238431.0

X: [1 1 1 1]
Y: [4 4 3 3 2 3 1 4 1 4]
Z: [0 1 5 0 3 4 2 5]

objValue: 1477861120001661.0

X: [1 1 1 1]
Y: [4 4 3 2 4 4 2 4 2 2]
Z: [0 1 5 0 3 4 2 5]

objValue: 7.785878611200083e+17

X: [1 1 1 1]
Y: [4 4 3 2 3 3 1 4 1 4]
Z: [0 1 5 0 3 4 2 5]

objValue: 8.33413341112012e+18

X: [1 1 1 1]
Y: [4 4 3 2 1 2 2 2 4 2]
Z: [0 1 5 0 3 4 2 5]

objValue: 8.333355633486679e+18

X: [1 1 1 1]
Y: [4 4 3 1 1 1 3 2 1 

KeyboardInterrupt: 